파인튜닝된 huggingface 기반 모델을 faster whisper로 바꾸는 코드

In [ ]:
# ★★★★★★★★★★ 1. 합칠 어댑터 경로 입력(이건 반드시 수정) ★★★★★★★★★★
# 보통 /home/kist/fast_whisper/가_ipynb_collection에 있는 수정본.ipynb 파일로 파인튜닝 시키고 그 결과로 adapter가 생성되기 때문에 대부분은 이 경로 안에 있을 것임.
# peft_model_id_offline = '[adapter_config.json, adapter_model.safetensors, README.md 가 있는 디렉토리]'
peft_model_id_offline = '/home/kist/fast_whisper/가_ipynb_collection/03261858'

# base model 경로(이거는 이 컴퓨터에서 위치 바꾸지 않는 이상 바꿀 필요 없음)
# base_model_name_or_path = '[config.json, tokenizer.json, 기타 모델 파일 등이 들어 있는 디렉토리]'
base_model_name_or_path = '/home/kist/fast_whisper/openai_whisperLargeV3/models--openai--whisper-large-v3/snapshots/1ecca609f9a5ae2cd97a576a9725bc714c022a93'

from transformers import (
    AutomaticSpeechRecognitionPipeline,
    WhisperForConditionalGeneration,
    WhisperTokenizer,
    WhisperProcessor,
)
from peft import PeftModel, PeftConfig
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import gc
import torch
import os
import time
from datetime import datetime
import shutil

#device = 'cpu'
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

language = "Korean"
task = "transcribe"

peft_config = PeftConfig.from_pretrained(peft_model_id_offline)



model = WhisperForConditionalGeneration.from_pretrained(
    base_model_name_or_path, load_in_8bit=False, device_map = 0
    ).to(device)

model = PeftModel.from_pretrained(model, peft_model_id_offline).to(device) #using adapter

# ---merging model and peft adapter---

now = datetime.now()
plus = now.strftime("%m%d%H%M")

merged_model = model.merge_and_unload()
merged_model.save_pretrained('merged_model_' + plus) # 이 경로에 adapter와 base model이 합쳐진 모델이 저장됨.

print('adapter 병합 완료!!!')

folder_name = './files_for_converting_to_faster_whisper'
filelist = os.listdir(folder_name)
dest_folder = '.'

for fl in filelist:
    shutil.copy(folder_name+'/'+fl, dest_folder+'/merged_model_'+plus+'/'+fl)

inst1 = 'ct2-transformers-converter --model merged_model_' + plus + ' --output_dir faster_merged_model_' + plus + ' --copy_files tokenizer.json preprocessor_config.json --quantization float16'
inst2 = 'rm -rf merged_model_' + plus
os.system(inst1)
os.system(inst2)

print('faster whisper 변환 완료!!!')




최종적으로 faster_merged_model_03261640과 같은 faster_whisper 기반의 모델 파일들을 담고 있는 디렉토리가 생성될 것임.